In [1]:
#OPTIONAL: uncomment to install packages if necessary

# %pip install pandas
# %pip install scikit-learn
# %pip install ta
# %pip install xgboost

Preprocessing the dataset

In [2]:
import pandas as pd

df_raw = pd.read_csv("crypto.csv")

df_raw.head()

,OpenDt,CRVUSDT-open,CRVUSDT-high,CRVUSDT-low,CRVUSDT-close,CRVUSDT-volume,ZILUSDT-open,ZILUSDT-high,ZILUSDT-low,ZILUSDT-close,...,EGLDUSDT-open,EGLDUSDT-high,EGLDUSDT-low,EGLDUSDT-close,EGLDUSDT-volume,LTCUSDT-open,LTCUSDT-high,LTCUSDT-low,LTCUSDT-close,LTCUSDT-volume
0,2020-09-04 06:00:00,2.962,2.975,2.800,2.814,233206.032,0.01617,0.01630,0.01588,0.01592,...,17.524,17.563,17.000,17.027,13078.587,48.77,48.77,48.20,48.21,23686.98714
1,2020-09-04 07:00:00,2.812,2.933,2.797,2.919,422883.332,0.01594,0.01599,0.01501,0.01591,...,17.029,17.900,16.710,17.129,24945.703,48.20,48.93,47.37,48.83,46079.69323
2,2020-09-04 08:00:00,2.919,3.237,2.915,3.171,599623.833,0.01592,0.01654,0.01588,0.01635,...,17.133,17.344,17.102,17.277,7945.829,48.78,49.92,48.75,49.67,73098.31532
3,2020-09-04 09:00:00,3.171,3.294,3.100,3.282,392916.735,0.01638,0.01689,0.01621,0.01684,...,17.277,17.277,16.918,17.121,15481.387,49.68,51.54,49.34,51.45,73369.67310
4,2020-09-04 10:00:00,3.272,3.278,3.150,3.157,374373.881,0.01683,0.01717,0.01672,0.01703,...,17.133,17.750,17.100,17.342,19422.335,51.42,52.38,51.00,51.42,81565.19476


In [3]:
# do not run this more than once without resetting kernel
def rename_cols(name):
    divider = name.index('-')
    return name[divider+1:] + "-" + name[:divider-4]

rename_map = {col: rename_cols(col) for col in df_raw.columns[1:]}

rename_map["OpenDt"] = "datetime"

df_raw.rename(columns=rename_map, inplace=True)

df_raw["datetime"] = pd.to_datetime(df_raw["datetime"])

df_raw.head()

,datetime,open-CRV,high-CRV,low-CRV,close-CRV,volume-CRV,open-ZIL,high-ZIL,low-ZIL,close-ZIL,...,open-EGLD,high-EGLD,low-EGLD,close-EGLD,volume-EGLD,open-LTC,high-LTC,low-LTC,close-LTC,volume-LTC
0,2020-09-04 06:00:00,2.962,2.975,2.800,2.814,233206.032,0.01617,0.01630,0.01588,0.01592,...,17.524,17.563,17.000,17.027,13078.587,48.77,48.77,48.20,48.21,23686.98714
1,2020-09-04 07:00:00,2.812,2.933,2.797,2.919,422883.332,0.01594,0.01599,0.01501,0.01591,...,17.029,17.900,16.710,17.129,24945.703,48.20,48.93,47.37,48.83,46079.69323
2,2020-09-04 08:00:00,2.919,3.237,2.915,3.171,599623.833,0.01592,0.01654,0.01588,0.01635,...,17.133,17.344,17.102,17.277,7945.829,48.78,49.92,48.75,49.67,73098.31532
3,2020-09-04 09:00:00,3.171,3.294,3.100,3.282,392916.735,0.01638,0.01689,0.01621,0.01684,...,17.277,17.277,16.918,17.121,15481.387,49.68,51.54,49.34,51.45,73369.67310
4,2020-09-04 10:00:00,3.272,3.278,3.150,3.157,374373.881,0.01683,0.01717,0.01672,0.01703,...,17.133,17.750,17.100,17.342,19422.335,51.42,52.38,51.00,51.42,81565.19476


In [4]:
df_raw.count()

df_raw.isnull().values.any()

np.False_

In [5]:
df = (
    pd.wide_to_long(
        df_raw,
        stubnames=["open", "high", "low", "close", "volume"],
        i="datetime",
        j="symbol",
        sep="-",
        suffix=".+",
    )
    .reset_index()
)

df = df.sort_values(['symbol','datetime']).set_index(['datetime'])

df

,symbol,open,high,low,close,volume
datetime,,,,,,
2020-09-04 06:00:00,ADA,0.09716,0.09766,0.09502,0.09502,22918282.0
2020-09-04 07:00:00,ADA,0.09506,0.09712,0.09215,0.09685,66824964.9
2020-09-04 08:00:00,ADA,0.09682,0.10012,0.09680,0.09944,44941018.0
2020-09-04 09:00:00,ADA,0.09945,0.10237,0.09886,0.10219,30595680.1
2020-09-04 10:00:00,ADA,0.10218,0.10392,0.10150,0.10227,44413004.8
...,...,...,...,...,...,...
2024-02-19 22:00:00,ZRX,0.36550,0.36740,0.36480,0.36640,316212.0
2024-02-19 23:00:00,ZRX,0.36680,0.36720,0.36330,0.36540,206017.0
2024-02-20 00:00:00,ZRX,0.36540,0.36780,0.36090,0.36240,543901.0


In [6]:
def ohlcv_4h(g):
    o = g['open'].resample('4H').first()
    h = g['high'].resample('4H').max()
    l = g['low' ].resample('4H').min()
    c = g['close'].resample('4H').last()
    v = g['volume'].resample('4H').sum()
    out = pd.DataFrame({'open':o,'high':h,'low':l,'close':c,'volume':v})
    return out.dropna()  # drop partial bars at edges if any


df = (
    df.groupby('symbol')
      .resample('4H')
      .agg(open=('open','first'),
           high=('high','max'),
           low =('low' ,'min'),
           close=('close','last'),
           volume=('volume','sum'))
      .dropna(subset=['open','high','low','close'])
      .reset_index()
      .set_index(['symbol','datetime'])
      .sort_index()
)

/var/folders/t4/r48zbdk509v7gbq_nv9spndw0000gn/T/ipykernel_78226/152338199.py:13: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .resample('4H')
/var/folders/t4/r48zbdk509v7gbq_nv9spndw0000gn/T/ipykernel_78226/152338199.py:14: FutureWarning: DataFrameGroupBy.resample operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .agg(open=('open','first'),


In [7]:
# Feature Engineering
from ta.momentum import RSIIndicator, StochasticOscillator, ROCIndicator
from ta.trend import EMAIndicator, MACD, CCIIndicator
from ta.volatility import BollingerBands
import numpy as np

c, h, l, v = df['close'], df['high'], df['low'], df['volume']


# ---- Params for hourly (4h params × 4)
# ROC_WINS   = [4, 12, 168]
# EMA_PAIRS  = [(336, 672)]
# RSI_WINS   = [24, 56, 104]
# CCI_WINS   = [40, 80]
# BB_WINS    = [40, 80]
# STO_KS     = [24, 56]
# VOL_WIN    = 80
# VOL_MA_VOL = 168
# VOL_ROC_W  = [168, 336]

# 4h params
ROC_WINS   = [1, 3, 42]
EMA_PAIRS  = [(84, 168)]
RSI_WINS   = [8, 14, 26]
CCI_WINS   = [10, 20]
BB_WINS    = [10, 20]
STO_KS     = [8, 14]
VOL_WIN    = 20
VOL_MA_VOL = 42
VOL_ROC_W  = [42, 84]

fe = pd.DataFrame(index=df.index)

# ROC
for n in ROC_WINS:
    fe[f'roc_{n}'] = ROCIndicator(close=c, window=n).roc()

# EMA cross (diff & ratio)
for s, L in EMA_PAIRS:
    ema_s = EMAIndicator(close=c, window=s).ema_indicator()
    ema_L = EMAIndicator(close=c, window=L).ema_indicator()
    fe[f'ema_diff_{s}_{L}']  = ema_s - ema_L
    fe[f'ema_ratio_{s}_{L}'] = ema_s / ema_L - 1.0

# RSI + lag2
for n in RSI_WINS:
    r = RSIIndicator(close=c, window=n).rsi()
    fe[f'rsi_{n}'] = r
    fe[f'rsi_{n}_lag2'] = r.shift(2)

# MACD histogram + lag2 (12,26,9)
macd_hist = MACD(close=c, window_fast=12, window_slow=26, window_sign=9).macd_diff()
fe['macd_hist'] = macd_hist
fe['macd_hist_lag2'] = macd_hist.shift(2)

# CCI
for n in CCI_WINS:
    fe[f'cci_{n}'] = CCIIndicator(high=h, low=l, close=c, window=n).cci()

# Bollinger %b and bandwidth
for n in BB_WINS:
    bb = BollingerBands(close=c, window=n, window_dev=2)
    fe[f'bb_pctb_{n}'] = bb.bollinger_pband()
    fe[f'bb_bw_{n}']   = bb.bollinger_wband()

# Stochastic (%K, %D, slowD) + histogram
for K in STO_KS:
    stoch = StochasticOscillator(high=h, low=l, close=c, window=K, smooth_window=3)
    fast_k = stoch.stoch()
    fast_d = stoch.stoch_signal()
    slow_d = fast_d.rolling(3, min_periods=3).mean()
    fe[f'stoch_fastk_{K}'] = fast_k
    fe[f'stoch_fastd_{K}'] = fast_d
    fe[f'stoch_slowd_{K}'] = slow_d
    fe[f'stoch_hist_{K}']  = fast_k - slow_d

# Price volatility: rolling std of log returns
logret = np.log(c / c.shift(1))
fe[f'volatility_{VOL_WIN}'] = logret.rolling(VOL_WIN, min_periods=VOL_WIN).std()

# Volume features: net volume mean & volume change
    
fe[f'net_volume_{VOL_MA_VOL}'] = v.rolling(VOL_MA_VOL, min_periods=VOL_MA_VOL).mean()

def volume_log_change(v: pd.Series, n: int, eps: float = 1e-9) -> pd.Series:
    return np.log1p(v + eps) - np.log1p(v.shift(n) + eps)
    
for n in VOL_ROC_W:
    fe[f'vol_change_{n}'] = volume_log_change(v, n)

In [8]:
fe

roc_1     roc_3     roc_42  ema_diff_84_168  \
symbol datetime                                                              
ADA    2020-09-04 04:00:00       NaN       NaN        NaN              NaN   
       2020-09-04 08:00:00  4.615385       NaN        NaN              NaN   
       2020-09-04 12:00:00 -2.951046       NaN        NaN              NaN   
       2020-09-04 16:00:00  2.491610  4.057821        NaN              NaN   
       2020-09-04 20:00:00  0.773963  0.236873        NaN              NaN   
...                              ...       ...        ...              ...   
ZRX    2024-02-19 08:00:00 -0.444198  1.156559  13.986014         0.002362   
       2024-02-19 12:00:00  0.920245  0.555710  12.566096         0.002659   
       2024-02-19 16:00:00  0.165792  0.638534  11.813695         0.002952   
       2024-02-19 20:00:00  0.800000  1.896263  11.538462         0.003268   
       2024-02-20 00:00:00 -0.136836  0.828958  12.519272         0.003566   

                            ema_ratio_84_168      rsi_8  rsi_8_lag2  \
symbol datetime                                                       
ADA    2020-09-04 04:00:00               NaN        NaN         NaN   
       2020-09-04 08:00:00               NaN        NaN         NaN   
       2020-09-04 12:00:00               NaN        NaN         NaN   
       2020-09-04 16:00:00               NaN        NaN         NaN   
       2020-09-04 20:00:00               NaN        NaN         NaN   
...                                      ...        ...         ...   
ZRX    2024-02-19 08:00:00          0.007121  68.734054   73.275552   
       2024-02-19 12:00:00          0.008008  72.939046   73.584988   
       2024-02-19 16:00:00          0.008881  73.674737   68.734054   
       2024-02-19 20:00:00          0.009819  77.111905   72.939046   
       2024-02-20 00:00:00          0.010703  75.177793   73.674737   

                               rsi_14  rsi_14_lag2     rsi_26  ...  \
symbol datetime                                                ...   
ADA    2020-09-04 04:00:00        NaN          NaN        NaN  ...   
       2020-09-04 08:00:00        NaN          NaN        NaN  ...   
       2020-09-04 12:00:00        NaN          NaN        NaN  ...   
       2020-09-04 16:00:00        NaN          NaN        NaN  ...   
       2020-09-04 20:00:00        NaN          NaN        NaN  ...   
...                               ...          ...        ...  ...   
ZRX    2024-02-19 08:00:00  65.428551    67.597706  62.579404  ...   
       2024-02-19 12:00:00  67.923018    67.802387  64.082794  ...   
       2024-02-19 16:00:00  68.369890    65.428551  64.353594  ...   
       2024-02-19 20:00:00  70.508436    67.923018  65.655219  ...   
       2024-02-20 00:00:00  69.634263    68.369890  65.228138  ...   

                            stoch_slowd_8  stoch_hist_8  stoch_fastk_14  \
symbol datetime                                                           
ADA    2020-09-04 04:00:00            NaN           NaN             NaN   
       2020-09-04 08:00:00            NaN           NaN             NaN   
       2020-09-04 12:00:00            NaN           NaN             NaN   
       2020-09-04 16:00:00            NaN           NaN             NaN   
       2020-09-04 20:00:00            NaN           NaN             NaN   
...                                   ...           ...             ...   
ZRX    2024-02-19 08:00:00      91.360602     -9.445709       90.000000   
       2024-02-19 12:00:00      88.882896    -11.168610       89.682540   
       2024-02-19 16:00:00      84.833113     -3.690256       91.269841   
       2024-02-19 20:00:00      82.144012      5.586049       94.505495   
       2024-02-20 00:00:00      82.096321      0.538409       90.460526   

                            stoch_fastd_14  stoch_slowd_14  stoch_hist_14  \
symbol datetime                                                             
ADA    2020-09-04 04:00:00             NaN         

In [9]:
# Up/Down/Neutral Labels

# hourly threshold (paper used +/-1% for 4h, +/-1.5% daily). using something in range 0.3%-0.5% since we have hourly data
# LABEL_THRESHOLD_1H = 0.004

LABEL_THRESHOLD = 0.01

rp = np.log(df['close'].shift(-1) / df['close'])
y = pd.Series(0, index=df.index)         # neutral
y[rp >  LABEL_THRESHOLD] =  1            # up
y[rp < -LABEL_THRESHOLD] = -1            # down

# Keep only up/down
mask = y != 0
y = y[mask]

# make 0-1 scale instead of -1/1 (needed for roc_auc unless we use multiclass)
y = (y > 0).astype(int)

# Align & drop NaNs (due to warmup windows); last row has NaN label from shift(-1)
Xy = fe.join(y.rename('label')).dropna()
X = Xy.drop(columns='label')
y = Xy['label'].astype(int)

In [10]:
Xy

roc_1     roc_3     roc_42  ema_diff_84_168  \
symbol datetime                                                              
ADA    2020-10-02 00:00:00  2.152183  1.018761  16.711339         0.000941   
       2020-10-02 04:00:00 -5.351972 -3.472505   7.764893         0.000925   
       2020-10-02 08:00:00 -4.546893 -7.711138  -3.384944         0.000858   
       2020-10-02 20:00:00  0.097014  2.630416  -4.198906         0.000745   
       2020-10-03 00:00:00  1.949171  2.401298  -0.671493         0.000733   
...                              ...       ...        ...              ...   
ZRX    2024-02-17 04:00:00  2.204586  2.961208   8.082064        -0.000380   
       2024-02-17 08:00:00 -1.955709 -0.670163   7.167557        -0.000229   
       2024-02-17 12:00:00 -2.640070 -2.439741   4.011282        -0.000189   
       2024-02-17 16:00:00  1.687255 -2.933563   5.666875        -0.000084   
       2024-02-18 20:00:00  0.396488  0.311262  11.024115         0.001489   

                            ema_ratio_84_168      rsi_8  rsi_8_lag2  \
symbol datetime                                                       
ADA    2020-10-02 00:00:00          0.009978  50.235670   40.838184   
       2020-10-02 04:00:00          0.009802  33.867797   40.297030   
       2020-10-02 08:00:00          0.009100  26.063729   50.235670   
       2020-10-02 20:00:00          0.007911  35.701838   34.007220   
       2020-10-03 00:00:00          0.007780  43.773862   35.297694   
...                                      ...        ...         ...   
ZRX    2024-02-17 04:00:00         -0.001156  63.367862   58.480175   
       2024-02-17 08:00:00         -0.000696  51.079337   52.291051   
       2024-02-17 12:00:00         -0.000573  39.494440   63.367862   
       2024-02-17 16:00:00         -0.000255  47.897592   51.079337   
       2024-02-18 20:00:00          0.004501  67.229287   64.741044   

                               rsi_14  rsi_14_lag2     rsi_26  ...  \
symbol datetime                                                ...   
ADA    2020-10-02 00:00:00  52.049412    47.360220  54.162627  ...   
       2020-10-02 04:00:00  41.303154    47.009730  47.441350  ...   
       2020-10-02 08:00:00  35.038659    52.049412  42.980949  ...   
       2020-10-02 20:00:00  40.493815    39.551616  45.925395  ...   
       2020-10-03 00:00:00  44.823943    40.277410  48.194731  ...   
...                               ...          ...        ...  ...   
ZRX    2024-02-17 04:00:00  62.980162    60.280149  61.066020  ...   
       2024-02-17 08:00:00  54.664398    56.148065  55.992328  ...   
       2024-02-17 12:00:00  46.006107    62.980162  50.245971  ...   
       2024-02-17 16:00:00  51.186878    54.664398  53.344435  ...   
       2024-02-18 20:00:00  63.745577    62.234581  61.281430  ...   

                            stoch_hist_8  stoch_fastk_14  stoch_fastd_14  \
symbol datetime                                                            
ADA    2020-10-02 00:00:00     14.454404       55.312810       41.873552   
       2020-10-02 04:00:00    -30.479500        5.192308       31.621534   
       2020-10-02 08:00:00    -26.366129        5.407210       21.970776   
       2020-10-02 20:00:00     13.280489       21.295060       20.182466   
       2020-10-03 00:00:00     22.381521       33.377837       25.122385   
...                                  ...             ...             ...   
ZRX    2024-02-17 04:00:00     32.559292       80.733945       63.347648   
       2024-02-17 08:00:00     -7.906912       49.541284       58.868502   
       2024-02-17 12:00:00    -34.594613       16.317992       48.864407   
       2024-02-17 16:00:00     -1.857481       39.748954       35.202743   
       2024-02-18 20:00:00      0.125142       89.527027       88.679828   

                            stoch_slowd_14  stoch_hist_14  volatility_20  \
symbol datetime                                                            
ADA    2020-10-02 00:00:00       41.1093

In [11]:
fe.count()

roc_1               371615
roc_3               371613
roc_42              371574
ema_diff_84_168     371449
ema_ratio_84_168    371449
rsi_8               371609
rsi_8_lag2          371607
rsi_14              371603
rsi_14_lag2         371601
rsi_26              371591
rsi_26_lag2         371589
macd_hist           371583
macd_hist_lag2      371581
cci_10              371607
cci_20              371597
bb_pctb_10          371607
bb_bw_10            371607
bb_pctb_20          322428
bb_bw_20            371597
stoch_fastk_8       371609
stoch_fastd_8       371607
stoch_slowd_8       371605
stoch_hist_8        371605
stoch_fastk_14      371603
stoch_fastd_14      371601
stoch_slowd_14      371599
stoch_hist_14       371599
volatility_20       371596
net_volume_42       371575
vol_change_42       371574
vol_change_84       371532
dtype: int64

In [12]:
# Normalize data if not already normalized

from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import TimeSeriesSplit

num_cols = X.select_dtypes(np.number).columns.tolist()

# already bound 0-100 or 0-1
bounded_cols   = [c for c in num_cols if c.startswith(('rsi_', 'stoch_', 'bb_pctb_'))]
# log1p then get z-score
loggy_cols     = [c for c in num_cols if c.startswith(('net_volume_', 'vol_change_', 'bb_bw_', 'volatility_', 'ema_ratio_'))]
# z-score
the_rest       = sorted(list(set(num_cols) - set(bounded_cols) - set(loggy_cols)))

def signed_log1p(df):
    Z = df.copy()
    for c in Z.columns:
        x = Z[c].values
        Z[c] = np.sign(x) * np.log1p(np.abs(x))
    return Z

pre = ColumnTransformer(
    transformers=[
        ("bounded_passthrough", "passthrough", bounded_cols),
        ("log_then_standardize", Pipeline([
            ("log",       FunctionTransformer(signed_log1p, validate=False)),
            ("standard",  StandardScaler())
        ]), loggy_cols),
        ("standardize_rest", StandardScaler(), the_rest)
    ],
    remainder="drop"
)


In [13]:
# Train-Test Split and Model Training

used_coins = []
skipped_coins = []

DESIRED_SPLITS = 5

for sym, g in X.groupby('symbol', sort=False):

    n = len(g)
    max_splits = (n - 200) // 540   # ensure ~200 training rows for lookback window
    actual_splits = min(8, max_splits)

    # use coins that have a reasonable amount of data
    if actual_splits < DESIRED_SPLITS:
        skipped_coins.append(sym)
        continue

    used_coins.append(sym)
    
    # tscv = TimeSeriesSplit(n_splits=actual_splits, test_size=540)
    
    # for train_idx, test_idx in tscv.split(g):
    #     X_train, X_test = g.iloc[train_idx], g.iloc[test_idx]
    #     y_train, y_test = y.loc[X_train.index], y.loc[X_test.index]

In [14]:
print("Used:", used_coins)
print("\n")
print("Skipped:", skipped_coins)

Used: ['ADA', 'ALGO', 'ATOM', 'BAL', 'BAND', 'BAT', 'BCH', 'COMP', 'CRV', 'DASH', 'DOT', 'EGLD', 'EOS', 'ETC', 'ICX', 'IOTA', 'KAVA', 'KNC', 'LINK', 'LTC', 'MKR', 'NEO', 'OMG', 'ONT', 'QTUM', 'RLC', 'RUNE', 'SNX', 'SOL', 'SUSHI', 'SXP', 'THETA', 'TRB', 'WAVES', 'XMR', 'XTZ', 'YFI', 'ZEC', 'ZRX']


Skipped: ['BNB', 'BTC', 'DOGE', 'ETH', 'IOST', 'TRX', 'VET', 'XLM', 'XRP', 'ZIL']


In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

models = {
    "LogisticRegression": LogisticRegression(max_iter=1000, n_jobs=-1),
    "RandomForest": RandomForestClassifier(n_estimators=500, random_state=42, n_jobs=-1),
    "XGBoost": XGBClassifier(
        n_estimators=2500, learning_rate=0.01, max_depth=3,
        subsample=0.75, colsample_bytree=0.75, random_state=42, n_jobs=-1
    )
}


In [27]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from ta.momentum import RSIIndicator
import numpy as np

results = []

results_roi = []

def to_signal(p_up, buy_thr=0.50, sell_thr=0.45):
    signal = np.zeros_like(p_up, dtype=int)
    signal[p_up >= buy_thr] = 1
    signal[p_up <  sell_thr] = -1
    return signal

def roi_sim(test_df):
    commission = 0.001  # 0.1% per side

    equity = 1000.0
    position = 0        # +1 if long, 0 if flat
    roi_series = []
    
    for sig, r in zip(test_df['signal'], test_df['ret']):
        # update position
        if sig == 1 and position == 0:   # buy
            equity *= (1 - commission)
            position = 1
        elif sig == -1 and position == 1: # sell
            equity *= (1 - commission)
            position = 0
    
        # apply return if holding
        if position == 1:
            equity *= np.exp(r)
    
        roi_series.append(equity)
    
    test_df['equity'] = roi_series
    ROI_model = (equity - 1000) / 1000
    # print(f"ROI (model) = {ROI_model*100:.1f}%")

    return ROI_model


for sym, g in Xy.groupby('symbol', sort=False):

    closes_g = df.loc[g.index, 'close']                 
    closes_g = closes_g.sort_index(level='datetime')
    ret_next_full = np.log(closes_g).diff().shift(-1)

    if sym in skipped_coins:
        continue
    
    print(g.columns)
    g = g.sort_index(level='datetime')
    X_sym = g.drop(columns=['label'])
    y_sym = g['label']

    tscv = TimeSeriesSplit(n_splits=DESIRED_SPLITS, test_size=540)
    print(f"\n=== {sym} ===")

    for fold, (train_idx, test_idx) in enumerate(tscv.split(X_sym)):
        X_train, X_test = X_sym.iloc[train_idx], X_sym.iloc[test_idx]
        y_train, y_test = y_sym.iloc[train_idx], y_sym.iloc[test_idx]

        test_index = g.iloc[test_idx].index                  # MultiIndex (symbol, datetime)

        ret_next = ret_next_full.loc[test_index]

        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        for name, model in models.items():
            model.fit(X_train, y_train)

            # Predict probabilities (for buy/hold/sell logic)
            proba = model.predict_proba(X_test)[:, 1]
            preds = (proba >= 0.5).astype(int)

            acc = accuracy_score(y_test, preds)
            auc = roc_auc_score(y_test, proba)

            results.append({
                "symbol": sym,
                "fold": fold + 1,
                "model": name,
                "accuracy": acc,
                "auc": auc
            })

            test_df = pd.DataFrame(
                {'proba': proba, 'pred': preds},
                index=test_index
            )
            test_df['ret'] = ret_next.values 

            test_df['signal'] = to_signal(test_df['proba'])

            roi = roi_sim(test_df)

            results_roi.append({
                "symbol": sym,
                "fold": fold + 1,
                "model": name,
                "roi": roi
            })

            print(f"{name:20s} Fold {fold+1}: acc={acc:.3f}, auc={auc:.3f}")


        # baseline B&H

        closes = df.loc[test_index, 'close']

        p0 = closes.iloc[0]
        p1 = closes.iloc[-1]
        ROI_BH = (p1 - p0) / p0

        results_roi.append({
                "symbol": sym,
                "fold": fold + 1,
                "model": "B&H",
                "roi": ROI_BH
            })
        
        # print(f"ROI (Buy & Hold) = {ROI_BH*100:.1f}%")

        # baseline STS
        
        rsi = RSIIndicator(closes, window=14).rsi()
        
        def rsi_sts(rsi, overbought=70, oversold=30):
            signal = np.zeros_like(rsi, dtype=int)
            signal[rsi < oversold]  = 1   # buy
            signal[rsi > overbought] = -1 # sell
            return signal

        sts_df = pd.DataFrame({
            'ret'  : ret_next.values
        })
        
        sts_df['signal'] = rsi_sts(rsi)

        ROI_STS = roi_sim(sts_df)

        results_roi.append({
            "symbol": sym,
            "fold": fold + 1,
            "model": "STS",
            "roi": ROI_STS
        })

        # print(f"ROI (STS) = {ROI_STS*100:.1f}%")
        

Index(['roc_1', 'roc_3', 'roc_42', 'ema_diff_84_168', 'ema_ratio_84_168',
       'rsi_8', 'rsi_8_lag2', 'rsi_14', 'rsi_14_lag2', 'rsi_26', 'rsi_26_lag2',
       'macd_hist', 'macd_hist_lag2', 'cci_10', 'cci_20', 'bb_pctb_10',
       'bb_bw_10', 'bb_pctb_20', 'bb_bw_20', 'stoch_fastk_8', 'stoch_fastd_8',
       'stoch_slowd_8', 'stoch_hist_8', 'stoch_fastk_14', 'stoch_fastd_14',
       'stoch_slowd_14', 'stoch_hist_14', 'volatility_20', 'net_volume_42',
       'vol_change_42', 'vol_change_84', 'label'],
      dtype='object')

=== ADA ===
LogisticRegression   Fold 1: acc=0.522, auc=0.556
RandomForest         Fold 1: acc=0.520, auc=0.540
XGBoost              Fold 1: acc=0.522, auc=0.528
LogisticRegression   Fold 2: acc=0.502, auc=0.499
RandomForest         Fold 2: acc=0.507, auc=0.522
XGBoost              Fold 2: acc=0.500, auc=0.517
LogisticRegression   Fold 3: acc=0.502, auc=0.492
RandomForest         Fold 3: acc=0.537, auc=0.538
XGBoost              Fold 3: acc=0.519, auc=0.534
Logisti

In [22]:
# Average by model
df_results = pd.DataFrame(results)

model_avgs = (
    df_results.groupby('model')[['accuracy', 'auc']]
    .mean()
    .sort_values('auc', ascending=False)
)
print(model_avgs.round(3))

                    accuracy    auc
model                              
LogisticRegression     0.520  0.528
RandomForest           0.517  0.524
XGBoost                0.515  0.523


In [36]:
# Look at ROI results by model by coin
df_results_roi = pd.DataFrame(results_roi)

model_roi_avgs = (
    df_results_roi.groupby(['model', 'symbol'])['roi']
    .mean()
    .sort_values(ascending=False)
)
print(model_roi_avgs.round(3))

model               symbol
B&H                 TRB       0.708
RandomForest        ONT       0.596
XGBoost             ONT       0.568
LogisticRegression  RLC       0.532
B&H                 SOL       0.525
                              ...  
LogisticRegression  THETA    -0.372
XGBoost             THETA    -0.422
                    DOT      -0.445
RandomForest        OMG      -0.458
LogisticRegression  OMG      -0.521
Name: roi, Length: 195, dtype: float64


In [43]:
roi_table = (
    df_results_roi
    .pivot_table(index='model', columns='symbol', values='roi', aggfunc='mean')
    .round(3)           # round to 3 decimals
    .sort_index(axis=0) # sort models alphabetically (optional)
)
roi_table['avg_roi'] = roi_table.mean(axis=1)

from IPython.display import display
display(roi_table.style
    .background_gradient(
        cmap='RdYlGn', 
        axis=None, 
        vmin=-roi_table.abs().max().max(),
        vmax= roi_table.abs().max().max()))

symbol,ADA,ALGO,ATOM,BAL,BAND,BAT,BCH,COMP,CRV,DASH,DOT,EGLD,EOS,ETC,ICX,IOTA,KAVA,KNC,LINK,LTC,MKR,NEO,OMG,ONT,QTUM,RLC,RUNE,SNX,SOL,SUSHI,SXP,THETA,TRB,WAVES,XMR,XTZ,YFI,ZEC,ZRX,avg_roi
model,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
B&H,-0.032000,-0.003000,0.056000,-0.257000,-0.056000,0.046000,-0.007000,-0.180000,-0.144000,-0.253000,-0.242000,0.391000,-0.307000,0.088000,-0.278000,0.073000,-0.331000,-0.146000,0.230000,-0.153000,0.177000,-0.050000,-0.349000,0.416000,0.037000,0.245000,0.151000,0.038000,0.525000,-0.146000,-0.224000,-0.180000,0.708000,-0.302000,-0.102000,-0.058000,-0.274000,-0.182000,-0.242000,-0.033769
LogisticRegression,-0.282000,-0.253000,-0.145000,-0.106000,0.076000,0.044000,-0.316000,-0.014000,0.100000,-0.333000,-0.318000,0.346000,-0.335000,0.046000,0.121000,-0.201000,-0.098000,-0.023000,-0.134000,-0.065000,0.004000,-0.042000,-0.521000,0.312000,-0.201000,0.532000,-0.351000,0.103000,0.139000,-0.232000,-0.245000,-0.372000,0.231000,-0.353000,-0.030000,-0.265000,-0.252000,-0.171000,-0.153000,-0.096333
RandomForest,-0.150000,-0.093000,0.234000,-0.158000,0.442000,-0.140000,-0.195000,0.107000,-0.010000,-0.169000,-0.316000,-0.017000,-0.201000,-0.067000,-0.096000,-0.105000,-0.241000,-0.257000,-0.240000,-0.106000,-0.107000,-0.035000,-0.458000,0.596000,0.039000,-0.014000,0.010000,-0.227000,0.106000,-0.054000,0.113000,-0.127000,0.134000,-0.048000,-0.074000,-0.193000,-0.064000,-0.234000,-0.308000,-0.069821
STS,0.035000,0.021000,-0.214000,0.121000,0.241000,0.060000,0.009000,-0.108000,-0.002000,0.068000,-0.055000,0.053000,-0.101000,0.072000,0.140000,-0.210000,-0.162000,-0.025000,0.235000,-0.096000,-0.073000,0.096000,-0.296000,0.270000,0.190000,0.221000,-0.115000,0.093000,-0.019000,-0.087000,-0.028000,-0.004000,-0.092000,-0.360000,0.024000,0.026000,0.053000,0.023000,-0.020000,-0.000410
XGBoost,-0.087000,0.185000,-0.016000,-0.208000,0.324000,-0.133000,-0.202000,-0.072000,-0.213000,-0.313000,-0.445000,0.035000,-0.285000,-0.150000,-0.116000,-0.156000,-0.315000,-0.200000,0.060000,-0.028000,0.087000,-0.122000,-0.207000,0.568000,0.063000,0.042000,0.030000,-0.351000,0.315000,-0.024000,0.035000,-0.422000,-0.085000,-0.218000,0.058000,0.082000,0.104000,-0.149000,-0.248000,-0.071205
